In [34]:
import pandas as pd

In [35]:
mongodbSinglePath = r"C:\Users\Felix Medicus\Desktop\benchmark_experiments\MongoDB\single\2.mongo_benchmark_execution_logs_spatiotemporal.txt"
mobilitydbSinglePath = r"C:\Users\Felix Medicus\Desktop\benchmark_experiments\MobilityDB\single\2.sql_benchmark_execution_logs_spatiotemporal.txt"

mobilitydbClusterPath = r"C:\Users\Felix Medicus\Desktop\benchmark_experiments\MobilityDB\cluster\1.mongo_benchmark_execution_logs_spatiotemporal.txt"
mobilitydbClusterPath = r"C:\Users\Felix Medicus\Desktop\benchmark_experiments\MobilityDB\cluster\1.sql_benchmark_execution_logs_spatiotemporal.txt"


In [36]:
df=pd.read_csv(mobilitydbSinglePath, skiprows=1)
print(df.columns)
df.head()

Index(['threadName', 'queryName', 'queryType', 'parameterValues', 'round',
       'executionIndex', 'startFirstQuery', 'endFirstQuery', 'startSecQuery',
       'endSecQuery', 'latency', 'fetchedRecords'],
      dtype='object')


,threadName,queryName,queryType,parameterValues,round,executionIndex,startFirstQuery,endFirstQuery,startSecQuery,endSecQuery,latency,fetchedRecords
0,thread-8,flightsInCountyInPeriod,spatiotemporal,tstzspan'[2023-01-25 04:50:28/ 2023-02-06 10:5...,0,0,1738969113545,1738969167683,0,0,54138,-1
1,thread-7,flightsInCountyInPeriod,spatiotemporal,tstzspan'[2023-08-19 01:06:09/ 2023-08-24 13:3...,0,0,1738969113544,1738969170179,0,0,56635,-1
2,thread-15,flightsInCountyInPeriod,spatiotemporal,tstzspan'[2023-11-25 22:00:48/ 2023-12-04 04:3...,0,0,1738969113545,1738969177725,0,0,64180,-1
3,thread-11,flightsInCountyInPeriod,spatiotemporal,tstzspan'[2023-08-16 20:51:36/ 2023-08-22 09:2...,0,0,1738969113544,1738969203263,0,0,89719,-1
4,thread-6,flightsInCountyInPeriod,spatiotemporal,tstzspan'[2023-11-28 04:48:32/ 2023-12-08 23:5...,0,0,1738969113545,1738969251702,0,0,138157,-1


In [37]:
spatiotemporal_df = df[df['queryType'] == 'spatiotemporal']
average_latency = spatiotemporal_df['latency'].mean()
print(f"Average Latency for spatiotemporal queries: {average_latency}")

Average Latency for spatiotemporal queries: 54083.863333333335


In [38]:
spatial_df = df[df[' queryType'] == 'spatial']
average_latency = spatial_df['latency'].mean()
print(f"Average Latency for spatial queries: {average_latency}")

KeyError: ' queryType'

In [ ]:
temporal_df = df[df[' queryType'] == 'temporal']
average_latency = temporal_df['latency'].mean()
print(f"Average Latency for temporal queries: {average_latency}")